## Machine Translation - Wikipedia Biographies - English to Spanish

Project Objective: To tune a model for translation from English to Spanish

Dataset Source: https://www.kaggle.com/datasets/paultimothymooney/translated-wikipedia-biographies?select=Translated+Wikipedia+Biographies+-+EN_ES.csv

##### Import Necessary Libraries

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import pyspark.sql.functions as F

##### Start Spark NLP Session

In [0]:
spark = sparknlp.start(gpu=True)

##### Ingest & Start Preprocessing Data

In [0]:
# File location and type
file_location = "/FileStore/tables/Translated_Wikipedia_Biographies___EN_ES.tsv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.drop('sourceLanguage', 'targetLanguage', 'documentID', 'stringID', 'entityName', 'sourceURL')

display(df)

sourceText,translatedText,perceivedGender
"Kaisa-Leena Mäkäräinen (born 11 January 1983) is a Finnish former world-champion and 3-time world-cup-winning biathlete, who currently competes for Kontiolahden Urheilijat.","Kaisa-Leena Mäkäräinen (nacida el 11 de enero de 1983) es excampeona mundial finlandesa, tres veces ganadora de la copa mundial de biatlón y actualmente compite para el Kontiolahden Urkeilijat.",Female
"Outside sports, Mäkäräinen is currently studying to be a Physics teacher at the University of Eastern Finland in Joensuu.","Además de los deportes, estudia actualmente en la Universidad de Finlandia Oriental en Joensuu para ser profesora de física.",Female
"Her team coach is Jonne Kähkönen, while Jarmo Punkkinen is her ski coach.","El entrenador de su equipo es Jonne Kähkönen, mientras que Jarmo Punkkinen es su entrenador de esquí.",Female
Mäkäräinen was originally a cross-country skier and focused on this until the age of twenty.,"Mäkäräinen era originalmente esquiadora de campo traviesa, a lo que se dedicó hasta la edad de 20 años.",Female
She started training for the biathlon in 2003.,Comenzó a entrenar para el biatlón en 2003.,Female
"In 2004, she made the Finnish National Team.",En 2004 fue parte del equipo nacional finlandés.,Female
2005 saw Mäkäräinen compete at the Biathlon World Championships for the first time.,En 2005 compitió por primera vez en el Campeonato Mundial de Biatlón.,Female
"Her best placings in the Biathlon World Championships have occurred at the 2007 World Championships in Antholz-Anterselva, where she placed eighth in the individual 15 and seventh in the mass start events.","Su mejor clasificación en el Campeonato Mundial de Biatlón se produjo en 2007 en Antholtz-Anterselva, donde quedó octava en la prueba individual 15, y séptima en la competencia de salida en masa.",Female
"At the 2008 Biathlon World Championships in Östersund, she was 15th in the mass start.",En el Campeonato Mundial de Biatlón de 2008 en Östersund quedó en el puesto 15 en la salida en masa.,Female
"During the 2007-08 Biathlon World Cup, she made it to the podium twice, the first time when she placed second in the sprint at Pokljuka, Slovenia and third in the pursuit at Ruhpolding, Germany.","Durante la Copa Mundial de Biatlón 2007/2008 logró subir al podio en dos ocasiones, la primera de ellas cuando obtuvo el segundo lugar en el sprint de Pokljuka, Eslovenia, y el tercero en la competencia de persecución en Ruhpolding, Alemania.",Female


##### Return Number of Samples in Dataset

In [0]:
df.count()

Out[16]: 1471

##### Return Unique Values in 'perceivedGender' Features (& Number of Unique Values)

In [0]:
unique_label_vals = df.select('perceivedGender').distinct().count()
print(unique_label_vals)
print(df.select('perceivedGender').distinct().show(unique_label_vals))

4
+-------------------+
|    perceivedGender|
+-------------------+
|Milikini Failautusi|
|             Female|
|            Neutral|
|               Male|
+-------------------+

None


##### Filter Columns to Remove Incorrect Samples

In [0]:
genders = ['Female', 'Male']

df = df.filter(df.perceivedGender.isin(genders))

unique_label_vals = df.select('perceivedGender').distinct().count()
print(unique_label_vals)
print(df.select('perceivedGender').distinct().show(unique_label_vals))

2
+---------------+
|perceivedGender|
+---------------+
|         Female|
|           Male|
+---------------+

None


##### Return Total Number of Samples in Processed Dataset & Drop Unnecessary Feature

In [0]:
df = df.drop('perceivedGender')
df.count()

Out[19]: 1344

##### Split Dataset into Training & Testing Datasets

In [0]:
train_ds, test_ds = df.randomSplit(weights=[0.80, 0.20], seed=42)

##### Define Pipeline Stages & Pipeline

In [0]:
doc = DocumentAssembler()\
    .setInputCol("sourceText")\
    .setOutputCol("document")

sentence = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

translator = MarianTransformer.pretrained("opus_mt_en_es", "xx")\
    .setInputCols(["sentence"])\
    .setOutputCol("translation")

en_es_translation_pipeline = Pipeline().setStages([doc, sentence, translator])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_en_es download started this may take some time.
Approximate size to download 398.8 MB
[OK!]


##### Fit/Train Model

In [0]:
en_es_translation_model = en_es_translation_pipeline.fit(train_ds)

##### Inference: Predictions Using Test Dataset

In [0]:
preds = en_es_translation_model.transform(test_ds)

##### Return Only Necessary Features & Convert to Pandas DataFrame

In [0]:
preds_in_pandas = preds.select(F.col("sourceText").alias("source"), F.col("translatedText").alias("ground_truth"), F.col("translation.result").alias("predictions")).toPandas()

##### Display Condensed Predictions Output

In [0]:
display(preds_in_pandas)

source,ground_truth,predictions
"""All they have to do is just support them in terms of looking for money, looking for food, and looking after the babies and the families"""".""","""Solo pueden acompañarlos en el sentido de que deben ganar dinero, procurar el alimento y cuidar a los bebés"""".""","List(""Todo lo que tienen que hacer es apoyarlos en términos de buscar dinero, buscar comida y cuidar a los bebés y las familias""., "")"
"""During this time she worked heavily with street children and was nicknamed """"Aunt Ruth"""", a name which continued to be used throughout her life.""","""Por entonces, realizó un arduo trabajo con niños de la calle y la llamaron """"tía Ruth"""", apodo que se siguió usando por el resto de su vida.""","List(""Durante este tiempo trabajó mucho con niños de la calle y fue apodada ""Tía Ruth"", un nombre que continuó siendo usado durante toda su vida., "")"
"""From 1934 to 1936, he lived in Alcobaça, where he went to study French and """"good manners"""".""","""Entre 1934 y 1936, vivió en Alcobaça, donde estudió francés y """"buenos modales"""".""","List(""De 1934 a 1936, vivió en Alcobaça, donde fue a estudiar francés y ""buenas costumbres""., "")"
"""Heavensgate (1962) opens with the lines:Before you, mother Idoto,naked I stand,while in """"Distances"""" (1964), he celebrates his final aesthetic and psychic return to his indigenous religious roots:I am the sole witness to my homecoming.""","""Su libro Puerta del cielo (1962) comienza con la frase """"Frente a ti, madre Idoto, estoy desnudo;..."""", mientras que en """"Distancias"""" (1964) celebra el retorno estético y psíquico final a sus raíces religiosas indígenas: """"Soy el único testigo de mi regreso a casa"""".""","List(""Heavensgate (1962) abre con las líneas:Antes de ti, madre Idoto, desnuda me paro, mientras que en ""Distancias"" (1964),, "")"
"""In 2006 he choreographed The Wedding for the Royal New Zealand Ballet, and collaborated with Amanda Eyles to restage Andrée Howard's 1939 work """"Lady into Fox"""".""","En 2006 coreografió “The Wedding” para el New Zeland Royal Ballet, y colaboró con Amanda Eyles para volver a representar la obra de Andrée Howard de 1939 “Lady into Fox”.","List(En 2006 coreografió The Wedding para el Royal New Zealand Ballet, y colaboró con Amanda Eyles para reafirmar la obra de Andrée Howard en 1939 ""Lady, "")"
"""In 2009 he voiced his support for the Crimes (Substituted Section 59) Amendment Act 2007 (the so-called """"anti-smacking"""" law) in advance of the citizens-initiated referendum later that year, aimed at repealing the act.""","""En 2009, dio a conocer su apoyo a la Enmienda de la Ley Criminal de 2007 (sustitución del artículo 59, también citada como ley """"antigolpes"""") como preámbulo del referendo iniciado por la ciudadanía posteriormente, ese mismo año, con el objetivo de rechazar la ley.""","List(""En 2009 expresó su apoyo a la Ley de Enmienda de Delitos (artículo 59) de 2007 (la llamada ley ""anti-smacking"") antes de la, "")"
"""In October 2017, Emily Orwaru was named among the """"Top 40 Under 40 Women in Kenya 2017"""" by Business Daily Africa, an English-language business daily newspaper, published by the Nation Media Group.""","En octubre de 2017, Emily Orwaru figuró en la lista de 40 mujeres de menos de 40 años más destacadas de Kenia en 2017 de Business Daily Africa, un diario empresarial en inglés publicado por Nation Media Group.","List(""En octubre de 2017, Emily Orwaru fue nombrada entre los """"Top 40 Under 40 Women in Kenya 2017"""" por Business Daily Africa, un diario de negocios en inglés,, "")"
"""Margarette """"Peggy"""" Golding (born Margarette Owen; November 1881 – 1939) was a Welsh born nurse and businessperson who was the founder of the """"Inner Wheel"""" club in Manchester that has grown to be an international organisation open to the wives of members of the Rotary Club.""","""Margarette “Peggy” Golding (nacida Margarette Owen; noviembre de 1881 – 1939) fue una enferm

##### Save Model

In [0]:
en_es_translation_model.save('/Users/briandunn/Desktop/NLP-Machine-Translation/English_to_Spanish-Translator-Model')

## Note(s) & Other Takeaways From This Project

****
- Unfortunately, I was unable to get a suitable solution to evaluate the results of this model. I tried a few different ideas and did some research to find a solution, but none of them worked to my liking.
****